In [2]:
import pandas as pd
import numpy as np

In [3]:
# create df
column_names = ["Date Time", "PM10", "PM2.5", "NO2", "SO2", "CO"]
df = pd.read_csv("gios-pjp-data.csv", sep=",", skiprows=1, header=None)
df = df.iloc[:, :-1]
df.columns = column_names
df[column_names[1:]] = df[column_names[1:]].replace(r'^\s*$', np.nan, regex=True)
df["Date Time"] = pd.to_datetime(df["Date Time"], format="%Y-%m-%d %H:%M")
df["PM10"] = df["PM10"].astype(float)
df["PM2.5"] = df["PM2.5"].astype(float)
df["NO2"] = df["NO2"].astype(float)
df["SO2"] = df["SO2"].astype(float)
df["CO"] = df["CO"].astype(float)
df.head(100)

,Date Time,PM10,PM2.5,NO2,SO2,CO
0,2023-10-01 01:00:00,18.9,25.8,16.3,2.1,0.2
1,2023-10-01 02:00:00,18.9,25.2,15.7,1.2,0.2
2,2023-10-01 03:00:00,19.2,25.3,14.5,NaN,0.2
3,2023-10-01 04:00:00,21.9,28.5,NaN,2.0,0.2
4,2023-10-01 05:00:00,21.0,25.5,17.1,2.3,NaN
...,...,...,...,...,...,...
95,2023-10-05 00:00:00,16.9,20.9,34.7,1.8,0.2
96,2023-10-05 01:00:00,18.3,23.5,32.5,2.1,0.2
97,2023-10-05 02:00:00,16.8,19.5,22.0,2.5,0.2
98,2023-10-05 03:00:00,17.3,19.9,20.6,2.1,0.2


In [5]:
df.dtypes

Date Time    datetime64[ns]
PM10                float64
PM2.5               float64
NO2                 float64
SO2                 float64
CO                  float64
dtype: object